In [ ]:
import argparse
import os

import numpy as np
import yaml

from config import (SPECS_COMPLETE_YAMLS, SPSOL_ROMOD_RESULTS_DIR, ROSOL_SPMOD_RESULTS_DIR,
                    SP_RESULTS_DIR, RO_RESULTS_DIR)
from model import DC, LPAC, Solution
from util import in_notebook

In [ ]:
# user inputs
if in_notebook():
    f_min = 0
    f_max = 5
    f_inc = 1
    r_hat = 3
    casestudy = 'imelda'
    pftype = 'dc'
else:
    parser = argparse.ArgumentParser()
    parser.add_argument('--fmin')
    parser.add_argument('--fmax')
    parser.add_argument('--finc')
    parser.add_argument('--rhat')
    parser.add_argument('--casestudy')
    parser.add_argument('--pftype')
    args = parser.parse_args()
    f_min = int(args.fmin)
    f_max = int(args.fmax)
    f_inc = int(args.finc)
    r_hat = int(args.rhat)
    casestudy = str(args.casestudy)
    pftype = str(args.pftype)

In [ ]:
model_classes = {
    'dc': DC,
    'lpacc': LPAC,
    'lpacf': LPAC,
    'qpac': LPAC
}

In [ ]:
# SP solution into RO model

# load grid data
with open(SPECS_COMPLETE_YAMLS[casestudy, pftype]) as fh:
    specs = yaml.load(fh, Loader=yaml.Loader)
    specs['options']['approach'] = 'robust'
    for key, val in specs['r_hat'].items():
        specs['r_hat'][key] = min(val, r_hat)
    for key in list(specs['xi']):
        (k, r, omega) = key
        if r > r_hat:
            specs['xi'].pop(key)
    for k in specs['R']:
        specs['R'][k] = [i for i in range(1, min(max(specs['R'][k]), r_hat) + 1)]

# instantiate a model
MODEL = model_classes[pftype](**specs)

# set flags
MODEL.model.setParam('OutputFlag', 0)
MODEL.model.setParam('Threads', 4)
MODEL.model.setParam('Presolve', 2)
MODEL.model.setParam('Method', 2)
MODEL.model.setParam('IntFeasTol', 1e-8)
MODEL.update()

# solve
for f in np.arange(f_min, f_max + f_inc, f_inc):
    sol_sp_filename = os.path.join(SP_RESULTS_DIR, f'{casestudy}-{pftype}-f{f}-r{r_hat}.zip')
    sol_sp = Solution.from_zip(sol_sp_filename)
    MODEL.con_resource_hi.RHS = f
    for (k, r), indicator in sol_sp['x'].items():
        MODEL.x[k, r].Start = int(indicator > 0.5)
    MODEL.update()
    MODEL.solve()
    os.makedirs(SPSOL_ROMOD_RESULTS_DIR, exist_ok=True)
    sol_ro_filename = os.path.join(SPSOL_ROMOD_RESULTS_DIR, f'{casestudy}-{pftype}-f{f}-r{r_hat}.zip')
    sol_ro = Solution.from_solved_instance(MODEL)
    sol_ro.to_zip(sol_ro_filename, variables=['x', 'ObjBound', 'ObjVal', 'gamma', 'gamma_under', 'gamma_over'])
    print(f, MODEL.model.getObjective().getValue())

In [ ]:
# RO solution into SP model

# load grid data
with open(SPECS_COMPLETE_YAMLS[casestudy, pftype]) as fh:
    specs = yaml.load(fh, Loader=yaml.Loader)
    specs['options']['approach'] = 'stochastic'
    for key, val in specs['r_hat'].items():
        specs['r_hat'][key] = min(val, r_hat)
    for key in list(specs['xi']):
        (k, r, omega) = key
        if r > r_hat:
            specs['xi'].pop(key)
    for k in specs['R']:
        specs['R'][k] = [i for i in range(1, min(max(specs['R'][k]), r_hat) + 1)]

# instantiate a model
MODEL = model_classes[pftype](**specs)

# set flags
MODEL.model.setParam('OutputFlag', 0)
MODEL.model.setParam('Threads', 4)
MODEL.model.setParam('Presolve', 2)
MODEL.model.setParam('Method', 2)
MODEL.model.setParam('IntFeasTol', 1e-8)
MODEL.update()

# solve
for f in np.arange(f_min, f_max + f_inc, f_inc):
    sol_ro_filename = os.path.join(RO_RESULTS_DIR, f'{casestudy}-{pftype}-f{f}-r{r_hat}.zip')
    sol_ro = Solution.from_zip(sol_ro_filename)
    MODEL.con_resource_hi.RHS = f
    for (k, r), indicator in sol_ro['x'].items():
        MODEL.x[k, r].Start = int(indicator > 0.5)
    MODEL.update()
    MODEL.solve()
    os.makedirs(ROSOL_SPMOD_RESULTS_DIR, exist_ok=True)
    sol_sp_filename = os.path.join(ROSOL_SPMOD_RESULTS_DIR, f'{casestudy}-{pftype}-f{f}-r{r_hat}.zip')
    sol_sp = Solution.from_solved_instance(MODEL)
    sol_sp.to_zip(sol_sp_filename, variables=['x', 'ObjBound', 'ObjVal', 'gamma', 'gamma_under', 'gamma_over'])
    print(f, MODEL.model.getObjective().getValue())